In [1]:
!pip install stable_baselines3[extra] -q
!pip install pyglet==1.5.27 -q
!pip install -U bposd -q
!pip install tensorflow -q # for tensorboard

In [2]:
# Append the common library for CPC codes
import os
import sys
# TODO: lets do something better here like refactor the common parts and different learning mech parts
sys.path.append(os.getcwd() + "/src")

## Setup the RL Env

In [3]:
import os
from stable_baselines3.common.env_checker import check_env
import gym
from gym import spaces
import numpy as np
import torch
import utils
from global_params import params
from scoring import score_dataset
from CPC import cpc_code, generate_random as gen_random_cpc


def flatten(l):
    return [item for sublist in l for item in sublist]


"""
Some quick thoughts:
-- Should we start with a specific code each time or always a new random code?
"""


class SwapLDPCEnv(gym.Env):
    """Custom Environment that follows gym interface"""
    metadata = {'render.modes': ['human']}

    def __init__(self, target_succ_rate=0.99):
        super(SwapLDPCEnv, self).__init__()

        self.target_succ_rate = target_succ_rate
        _, m_b, m_p, m_c = gen_random_cpc.random_cpc()
        self.m_b = m_b
        self.m_p = m_p
        self.m_c = m_c
        # self.target_succ_rate = target_succ_rate
        # Each action corresponds to choosing to parity checks and the corresponding edges to swap
        self.action_space = spaces.MultiDiscrete([
            3,  # select which matrix to operate on, m_b, m_p, or m_c
            # select which parity check to operate on
            params['n_data_qubits'],
            # higher than the check qubit index return a low reward
            params['n_check_qubits'],
            # select which data qubit to operate on. If m_c is selected, have choosing a data qubit
        ])
        self.last_fer = 0
        self.n_steps = 0

        self.n_qubits = n_qubits = params['n_data_qubits'] + params['n_check_qubits']
        flattened_pc_size = 2 * \
            (n_qubits) * \
            params['n_check_qubits']

		# The first n qubits represent the noise distribution
        # TODO: THIS ALLOWS US TO TRAIN FOR "ADAPTIVE NOISE!!" (i.e. lets decrease connections...)
        # The quantum parity check matrix
        self.observation_space = spaces.Box(low=0.0, high=1.0,
                                            shape=(n_qubits + flattened_pc_size,), dtype=np.float32)
        self.best_succ = 0

    def step(self, action):
        self.n_steps += 1
        p_fails = np.ones(self.n_qubits) * np.random.uniform(low=params['constant_error_rate_lower'], high=params['constant_error_rate_upper'])
        if action[0] == 0:
            self.m_b[action[1], action[2]] = 1 - self.m_b[action[1], action[2]]
        elif action[0] == 1:
            self.m_p[action[1], action[2]] = 1 - self.m_p[action[1], action[2]]
        elif action[0] == 2:
            if action[1] >= params['n_check_qubits']:
                old_code_pc = cpc_code.get_classical_code_cpc(self.m_b, self.m_p, self.m_c)
                flattened = np.array(old_code_pc).astype(np.float32).flatten()
                obs = np.concatenate((p_fails, flattened)).astype(np.float32)
                return obs, -1, False, {}  # Return a very low reward
            self.m_c[action[1], action[2]] = 1 - self.m_c[action[1], action[2]]
        else:
            raise "Undefined selector action"

        code_pc = cpc_code.get_classical_code_cpc(self.m_b, self.m_p, self.m_c)
        # TODO: p_fail??
        succ_rate = score_dataset.run_decoder(code_pc, p_fails)

        self.last_fer = 1 - succ_rate

        reward = succ_rate
        flattened = np.array(code_pc).astype(np.float32).flatten()
        obs = np.concatenate((p_fails, flattened)).astype(np.float32)
        return obs, reward, succ_rate >= self.target_succ_rate, {}

    def reset(self):
        p_fails = np.ones(self.n_qubits) * np.random.uniform(low=params['constant_error_rate_lower'], high=params['constant_error_rate_upper'])
        _, m_b, m_p, m_c = gen_random_cpc.random_cpc()
        self.m_b = m_b
        self.m_p = m_p
        self.m_c = m_c
        code_pc = cpc_code.get_classical_code_cpc(self.m_b, self.m_p, self.m_c)
        # reward, done, info can't be included
        npd = np.array(code_pc).astype(np.float32)
        return np.concatenate((p_fails, npd.flatten())).astype(np.float32)

    def render(self, mode='console'):
        pass

    def close(self):
        pass


env = SwapLDPCEnv()
check_env(env, warn=True)


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


...

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


## Begin Training

In [2]:
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_util import make_vec_env

env = SwapLDPCEnv() 
model_type = "PPO"
check_env(env, warn=True)
print(env.best_succ)

# wrap it
env = make_vec_env(lambda: env,   n_envs=1)
loading_saved = False
model = None
if not loading_saved:
	model = PPO("MlpPolicy", env=env, tensorboard_log=f"./logs/{model_type}-tensorboard")
else:
	# TODO!
	pass

NameError: name 'SwapLDPCEnv' is not defined

In [5]:
## Setup the callbacks
from stable_baselines3 import SAC
from stable_baselines3.common.callbacks import EvalCallback, CallbackList, BaseCallback
import tensorflow as tf
import numpy as np

class TensorboardCallback(BaseCallback):
    """
    Custom callback for plotting additional values in tensorboard.
    """
    def __init__(self, verbose=0):
        self.is_tb_set = False
        super(TensorboardCallback, self).__init__(verbose)

    def _on_step(self) -> bool:
        # Log additional tensor
        if not self.is_tb_set:
            with self.model.graph.as_default():
                tf.summary.scalar('value_target', tf.reduce_mean(self.model.value_target))
                self.model.summary = tf.summary.merge_all()
            self.is_tb_set = True
        # Log scalar value (here a random variable)
        fer = self.model.get_env().last_fer
        summary = tf.Summary(value=[tf.Summary.Value(tag='Frame Error Rate', simple_value=fer)])
        self.locals['writer'].add_summary(summary, self.num_timesteps)
        return True


# Use deterministic actions for evaluation
eval_callback = EvalCallback(env, best_model_save_path=utils.get_best_scoring_model_path_rl(),
                             log_path='./logs/rl-{model_type}-eval-logs', eval_freq=500,
                             deterministic=True, render=False)

callback = CallbackList([eval_callback, TensorboardCallback()])

2023-02-09 14:11:52.679094: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-09 14:11:53.464262: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lev/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-09 14:11:53.464344: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/lev/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023

In [6]:

model.learn(total_timesteps=250_000, callback=[eval_callback])

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt


aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0

aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002' --mdm-type OOK --mnt-max-fe 10000 > build/tmp_out.txt
aff3ct --sim-cde-type LDPC --chn-type BSC --enc-cw-size 102 --enc-info-bits 82 --enc-type LDPC_H --dec-h-path build/tmp_code.alist --dec-type BP_FLOODING --dec-implem AMS --dec-ite 10 --sim-noise-type EP --sim-noise-range '0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0.002,0